Clustering Neighborhoods in Toronto

In [70]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import folium
from sklearn.cluster import KMeans
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

Fetch Postal Code of Canada

res=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup=BeautifulSoup(res.content,'html.parser')
table=soup.find_all('table')
table
df=pd.read_html(str(table))[0]
df.head()

In [1]:
Remove the borough "Not Assigned"

SyntaxError: invalid syntax (<ipython-input-1-aa9c5e0348c2>, line 1)

In [96]:
dfb=df[df['Borough']!='Not assigned']
dfb.reset_index(drop=True)

#dfb['Borough'].drop_duplicates()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [32]:
dfn=dfb[dfb['Borough'].str.contains('York')]

In [33]:
dft=dfb[dfb['Borough'].str.contains('Toronto')]

In [34]:
dfg=pd.read_csv('C:/Users/User/Desktop/Coursera/Applied Datascience Capstone/Geospatial_Coordinates.csv')
dfg.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Retrieve latitude and longitude for each neighborhood

In [94]:
dfnm=pd.merge(left=dfb, right=dfg, how='left', left_on='Postal Code', right_on='Postal Code')

dfnm

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [ ]:
dftm=pd.merge(left=dft, right=dfg, how='left', left_on='Postal Code', right_on='Postal Code')
dftm

In [103]:
dfnm['Neighbourhood'].drop_duplicates()

0                                              Parkwoods
1                                       Victoria Village
2                              Regent Park, Harbourfront
3                       Lawrence Manor, Lawrence Heights
4            Queen's Park, Ontario Provincial Government
                             ...                        
98         The Kingsway, Montgomery Road, Old Mill North
99                                  Church and Wellesley
100    Business reply mail Processing Centre, South C...
101    Old Mill South, King's Mill Park, Sunnylea, Hu...
102    Mimico NW, The Queensway West, South of Bloor,...
Name: Neighbourhood, Length: 99, dtype: object

In [97]:
CLIENT_ID='AANOCUB4LUU11ISUOA3AWETXU5TAOJYZPKTFVTMJ1BUQJJH5'
CLIENT_SECRET='EZEMNZT22RLFGSSDOCLFSXIY13EUKJ1Y14QB2XOESYWMDLM3'
VERSION='20180605'
LIMIT=100
print ('Your Credentials:')
print ('CLIENT ID: ' + CLIENT_ID)
print ('CLIENT SECRET: ' + CLIENT_SECRET)

Your Credentials:
CLIENT ID: AANOCUB4LUU11ISUOA3AWETXU5TAOJYZPKTFVTMJ1BUQJJH5
CLIENT SECRET: EZEMNZT22RLFGSSDOCLFSXIY13EUKJ1Y14QB2XOESYWMDLM3


In [39]:
dfnm_latitude=dfnm.loc[33,'Latitude']
dfnm_longitude=dfnm.loc[33,'Longitude']
dfnm_name=dfnm.loc[33,'Neighbourhood']
print ('Latitude and longitude values of {} are {},{}.'.format(dfnm_name,dfnm_latitude,dfnm_longitude))

Latitude and longitude values of Willowdale, Willowdale West are 43.7827364,-79.4422593.


In [40]:
radius = 500
LIMIT = 100
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    dfnm_latitude,
    dfnm_longitude,
    radius,
    LIMIT
)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=AANOCUB4LUU11ISUOA3AWETXU5TAOJYZPKTFVTMJ1BUQJJH5&client_secret=EZEMNZT22RLFGSSDOCLFSXIY13EUKJ1Y14QB2XOESYWMDLM3&v=20180605&ll=43.7827364,-79.4422593&radius=500&limit=100'

In [ ]:
results=requests.get(url).json()
results

In [42]:
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
places=results['response']['groups'][0]['items']
nearby_places=json_normalize(places)
nearby_places
#nearby_places.loc[0,:]
#nearby_places.columns
columns=['venue.name', 'venue.location.address', 'venue.location.lat',
       'venue.location.lng', 'venue.location.distance', 'venue.location.postalCode',
       'venue.location.cc', 'venue.location.city', 'venue.location.state',
       'venue.location.country', 'venue.location.formattedAddress',
       'venue.categories', 'venue.photos.count', 'venue.photos.groups',
       'venue.venuePage.id', 'venue.location.crossStreet']
nearby_places=nearby_places.loc[:,columns]
#for col in nearby_places.columns:
nearby_places.columns=[col.split('.')[-1] for col in nearby_places.columns]
nearby_places['categories']=nearby_places.apply(get_category_type,axis=1)
nearby_places.head()

c:\users\user\appdata\local\programs\python\python37-32\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,name,address,lat,lng,distance,postalCode,cc,city,state,country,formattedAddress,categories,count,groups,id,crossStreet
0,Tov-Li,5982 Bathurst St.,43.784214,-79.446098,349,M2R 1Z1,CA,Toronto,ON,Canada,"[5982 Bathurst St. (at Rockford Rd.), Toronto ...",Pizza Place,0,[],NaN,at Rockford Rd.
1,Shoppers Drug Mart,5998 Bathurst St,43.784847,-79.446028,383,M2R 1Z1,CA,North York,ON,Canada,"[5998 Bathurst St (at Cedarcroft Blvd.), North...",Pharmacy,0,[],NaN,at Cedarcroft Blvd.
2,Dollarama,"5990 Bathurst St, Cedarcroft",43.784670,-79.446670,414,M2R 1Z1,CA,Toronto,ON,Canada,"[5990 Bathurst St, Cedarcroft, Toronto ON M2R ...",Discount Store,0,[],NaN,NaN
3,Tim Hortons,515 Drewry Ave,43.780940,-79.444231,255,M2R 2K9,CA,North York,ON,Canada,"[515 Drewry Ave (at Bathurst St.), North York ...",Coffee Shop,0,[],NaN,at Bathurst St.
4,Price Chopper,6201 Bathurst St.,43.783237,-79.446339,332,NaN,CA,Toronto,ON,Canada,"[6201 Bathurst St. (at Steeles Ave. W), Toront...",Grocery Store,0,[],NaN,at Steeles Ave. W


In [44]:
print ('{} venues were returned by Foursquare.'.format(nearby_places.shape[0]))

6 venues were returned by Foursquare.


Find the nearby places in 500 meter

In [99]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name,lat,lng in zip(names, latitudes, longitudes):
        print (name)
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        results=requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results]) 
        #for venue_list in venues_list:
            #for item in venue_list:
                #print (item)
        nearby_places=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_places.columns=['Neighborhood',
                              'Neighborhood Latitude',
                              'Neighborhood Longitute',
                              'Venue',
                              'Venue Latitude',
                              'Venue Longitute',
                              'Venue Category']
    return (nearby_places)

In [ ]:
#dftr.head()
dfnm_venues=getNearbyVenues(names=dfnm['Neighbourhood'],
                           latitudes=dfnm['Latitude'], 
                           longitudes=dfnm['Longitude'])

#dfnm_venues=getNearbyVenues(names=dfnm.loc[2,'Neighbourhood'],
#                           latitudes=dfnm.loc[2,'Latitude'], 
#                           longitudes=dfnm.loc[2,'Longitude'])
#dfnm.loc[2,'Longitude']
#dfnm.loc[2,'Neighbourhood']

In [106]:
#dfnm[dfnm['Neighbourhood']=='Willowdale, Willowdale West']
dfnm_venues
#dfnm_venues['Neighborhood'].drop_duplicates()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitute,Venue,Venue Latitude,Venue Longitute,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
...,...,...,...,...,...,...,...
2118,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Islington Florist & Nursery,43.630156,-79.518718,Flower Shop
2119,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,43.631370,-79.519006,Tanning Salon
2120,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Once Upon A Child,43.631075,-79.518290,Kids Store
2121,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Value Village,43.631269,-79.518238,Thrift / Vintage Store


In [102]:
dfnm_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitute,Venue,Venue Latitude,Venue Longitute,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
...,...,...,...,...,...,...
"Willowdale, Willowdale West",6,6,6,6,6,6
Woburn,3,3,3,3,3,3
Woodbine Heights,7,7,7,7,7,7


In [116]:
dfnm_onehot=pd.get_dummies(dfnm_venues[['Venue Category']], prefix="",prefix_sep="")
dfnm_onehot['Neighborhood']=dfnm_venues['Neighborhood']
fixed_columns=[dfnm_onehot.columns[-1]] + list(dfnm_onehot.columns[:-1])
dfnm_onehot=dfnm_onehot[fixed_columns]
#dfnm_onehot=dfnm_onehot[['Neighborhood']]+dfnm_onehot.iloc[1:]
#fnm_onehot.columns['Neighborhood']
#list(dfnm_onehot.columns[:-1])
dfnm_onehot.head()
#dfnm_onehot.loc[:,'Neighborhood']

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
dfnm_grouped=dfnm_onehot.groupby('Neighborhood').mean().reset_index()
dfnm_grouped[['Park','Food & Drink Shop','Construction & Landscaping']][dfnm_grouped['Neighborhood']=='Parkwoods']
             


,Park,Food & Drink Shop,Construction & Landscaping
19,0.5,0.5,0.0


In [51]:
top_places=5
for x in dfnm_grouped['Neighborhood']:
    print ("----"+x+"----")
    temp=dfnm_grouped[dfnm_grouped['Neighborhood']==x].T.reset_index()
    temp.columns=['venue','freq']
    temp=temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp=temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(top_places))
    print ('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0    Coffee Shop  0.09
1           Bank  0.09
2  Grocery Store  0.05
3    Bridal Shop  0.05
4           Park  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2  Japanese Restaurant  0.25
3                 Bank  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0       Italian Restaurant  0.12
1           Sandwich Place  0.08
2              Pizza Place  0.08
3              Coffee Shop  0.08
4  Comfort Food Restaurant  0.04


----Caledonia-Fairbanks----
               venue  freq
0               Park  0.50
1      Women's Store  0.25
2               Pool  0.25
3  Accessories Store  0.00
4       Liquor Store  0.00


----Del Ray, Mount Dennis, Keelsdale and Silverthorn----
                venue  freq
0  Turkish Restaurant  0.25
1          Restaurant  0.25
2      Sandwich Place  0.25
3

Find top 10 most common places

In [52]:
def most_common_places(row,top_places):
    row_categories=row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:top_places]

In [80]:
top_places=10
indicators=['st','nd','rd']
columns=['Neighborhood']
for ind in np.arange(top_places):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
#print (columns)
neighborhoods_venues_sorted=pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood']=dfnm_grouped['Neighborhood']
#print (neighborhoods_venues_sorted)
for ind in np.arange(dfnm_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:]=most_common_places(dfnm_grouped.iloc[ind,:],top_places)
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Gas Station,Diner,Middle Eastern Restaurant,Mobile Phone Shop,Park,Pet Store,Pharmacy,Pizza Place
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Yoga Studio,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Boutique,Comfort Food Restaurant,Café,Liquor Store,Juice Bar,Pet Store
3,Caledonia-Fairbanks,Park,Pool,Women's Store,Food Truck,Food Court,Clothing Store,Coffee Shop,Comfort Food Restaurant,French Restaurant,Construction & Landscaping
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Discount Store,Turkish Restaurant,Restaurant,Sandwich Place,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store


K means clustering

In [77]:
kclusters=5
dfnm_grouped_clustering=dfnm_grouped.drop('Neighborhood',1)
#dfnm_grouped_clustering
kmeans=KMeans(n_clusters=kclusters, random_state=0).fit(dfnm_grouped_clustering)
#kmeans
kmeans.labels_[0:10]
kmeans.inertia_
kmeans.cluster_centers_
kmeans.n_iter_

3

In [3]:
#neighborhoods_venues_sorted.insert(0,'Cluster Labels',kmeans.labels_)
dfnm_merged=dfnm
dfnm_merged.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
dfnm_merged=dfnm_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'),on='Neighborhood')
dfnm_merged['Cluster Labels'].replace(np.NaN,0,inplace=True)
dfnm_merged['Cluster Labels']=dfnm_merged['Cluster Labels'].astype(int)
dfnm_merged.head()
dfnm_merged['Cluster Labels'].drop_duplicates()
#dfnm_merged[dfnm_merged['Cluster Labels']==3]

NameError: name 'dfnm' is not defined

In [56]:
dfnm.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937


In [91]:
latitude=43.753259
longtitude=-79.329656
map_clusters=folium.Map(location=[latitude, longtitude], zoom_start=11)
x=np.arange(kclusters)
ys=[i+x+(i*x)**2 for i in range(kclusters)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]
print (rainbow)
markers_colors=[]
for lat, lon, poi, cluster in zip(dfnm_merged['Latitude'],dfnm_merged['Longitude'],dfnm_merged['Neighborhood'],dfnm_merged['Cluster Labels']):
    label=folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']
